In [92]:
from datetime import datetime
from requests import get as curl
import ujson as json
import pandas as pd
import numpy as np
from functools import reduce
import os
from numba import jit

## Plotting imports
import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import psycopg2 as postgre

In [93]:
coinsdb = postgre.connect("MY LOCAL DB CREDENTIALS")
cursor = coinsdb.cursor()
coinsdb.rollback() # Clear any previous errors.
# cursor.execute(f"SELECT symbols FROM symbols;")
# symbols = [x[0].lower() for x in cursor.fetchall() ]
def run(s):
    cursor.execute(s)
    print( cursor.fetchall() )

# List of strings with the names of all data tables in DB
cursor.execute(f"SELECT tables FROM tables;")
tables = [x[0].lower() for x in cursor.fetchall() ]

numDays = 7 # last 7 days
numrows = numDays * 24 # times hourly measurements 
numrows += 1 # add one more as the day0 measurement for percent changes

In [94]:
# We clean timestamps by rounding them to nearest hour, so that charts look nicer (every hour sharp) 
def clean_index(index):
    newIdx = []
    for idx in index:
        newIdx.append(pd.Timestamp(str( idx )[:13] + ":00:00"))
    return pd.DatetimeIndex(newIdx)

data = dict()  # dict of panda Dataframes, each entry is one table from coins.db
for table in tables:
    # QUERY all columns from this table
    cursor.execute(f"SELECT column_name FROM information_schema.columns WHERE table_name = '{table}';")
    colNames = [x[0] for x in cursor.fetchall() ]#eliminate timestamp from column names
    
    # Get most recent #numrows of data 
    cursor.execute(f"SELECT * FROM (SELECT * FROM {table} ORDER BY timestamp DESC LIMIT {numrows}) as rows ORDER BY timestamp ASC;")
    cols = cursor.fetchall()
    
    # Process to build dataframe with clean index. Using pandas.read_sql would read the entire db which is not feasible.
    # Another reason is we found a bug with read_sql where some tables would not load completely at sometimes.
    vals = dict()
    for i, colName in enumerate(colNames):
        if i == 0:
            idx = [col[0] for col in cols]
        else:
            vals[colName.upper()] = [col[i] for col in cols]
    
    data[table] = pd.DataFrame.from_dict(vals)
    data[table].index = clean_index( idx )

In [95]:
# Applies boolean function to cols or rows of df and returns those which return true.
def filterDF(df: pd.DataFrame, f, axis=0):
    res = []
    if axis in [0, 'columns']:
        for col in df:
            if f(col):
                res.append(col.name)
        return df.loc[: , res]
    elif axis in [1,'rows','index']:
        for row in df.index:
            if df.loc[row,:].apply(f):
                res.append(row.name)
        return df.loc[res , :]

# Return elements of df where f(elem) == True in a list
def filter_elements(df, f):
    res = []
    for row in df.index:
        for col in df.columns:
            val = df.loc[row, col]
            if f( val ):
                res.append([ row , col , val ])
    return res

# Retrieves the last n entries to analyze and cleans the entries of missing or 0-values and duplicates
# This cleaning is due to Binance's API failures. 
# We later find the common columns free of failures for all tables
def last(table, numrows):
    # Drop the date and time to leave only data points as entries
    df = data[table].tail(numrows).dropna(axis='columns')
    goodcols = df.apply(lambda col: (len(col.unique()) > len(col)/2 ) & ((col != 0).all()) )
    return df.loc[:, goodcols]

In [96]:
prices_raw = last('weightedavgprice', numrows)
volumes_raw = last('quotevolume', numrows)

startDate = prices_raw.index[0].date()
endDate = prices_raw.index[-1].date() # Most recent day in data. Should be today.

# Here we store the daily USD trading value for non-USD quote coins so each volume and price are shown relative to USD. 
# As of this writing, these are the only trading coins in Binance: ['BTC', 'ETH','BNB','TUSD','PAX','USDT', 'USDC']
# For simplicity, we don't convert USDT and TUSD pairs because:
#  1) a USDT-USDC or TUSD-USDC pair does not exist yet to facilitate conversion
#  2) since they are all USD-based stablecoins they should not have any difference.
endings = dict(BTC=prices_raw['BTCUSDC'], ETH=prices_raw['ETHUSDC'], XRP=prices_raw['XRPUSDC'],
               BNB=prices_raw['BNBUSDC'], PAX=prices_raw['PAXUSDT'], 
               SDT=1, USD=1, SDC=1)

# Function to convert all quoteVolumes to BTC-volumes for comparison. apply to volumesPlus1
def convert_USD(col):
    if col.name[:4] == 'usdc':
        return col
    else:
        return col * endings[col.name[-3:]]

# We have to find common columns because the source data is not clean nor correct among all tables.
common_cols = {x for x in prices_raw.columns if x[:4] != 'usdc'}.intersection( set(volumes_raw.columns) )

# Convert all values to USDC so we can standardize their comparison around their USD value.
pricesPlus1 = prices_raw.loc[:, common_cols].apply( convert_USD )
volumesPlus1 = volumes_raw.loc[:, common_cols].apply( convert_USD )

# Make list of top Volumes (standardized in USD) to filter out coins with high price fluctuations but low volume. 
topVols = volumesPlus1.loc[:, volumesPlus1.mean() >= volumesPlus1.mean().median() ].columns
excluded_pairs = list( set(data['quotevolume'].columns) - set(topVols) )

# baseVolume = volumesPlus1.iloc[0][topVols]
# basePrice = pricesPlus1.iloc[0][topVols]

# These are ready to graph df's
prices = pricesPlus1.iloc[1:][topVols]
volumes = volumesPlus1.iloc[1:][topVols]

correlationPrices = prices.corr()
correlationVolumes = volumes.corr()
correlationValues = volumes.corrwith(prices).sort_values(ascending=False)

# These df's contain daily percent change from day1 to day7. 
# With day1 being the change from day0 (basePrice/Vol). 
volumePercents = volumesPlus1[topVols].pct_change().iloc[1:] * 100
pricePercents = pricesPlus1[topVols].pct_change().iloc[1:] * 100

correlationVolumePercents = volumePercents.corr()
correlationPricePercents = pricePercents.corr()
correlationPercents = volumePercents.corrwith(pricePercents).sort_values(ascending=False)

# Log returns is a measure used in finance similar to percent change. 
def log_returns(df):
    return df.applymap(np.log).diff()[topVols].iloc[1:]

volumesLogReturn = log_returns(volumesPlus1)
pricesLogReturn = log_returns(pricesPlus1)

correlationVolumesLogReturn = volumesLogReturn.corr()
correlationPricesLogReturn = pricesLogReturn.corr()
correlationLogReturns = pricesLogReturn.corrwith(volumesLogReturn).sort_values(ascending=False)

# This finds the total percent change from day0 to day7 and result is in percent to display in graph directly.
def total_percent_change(df: pd.DataFrame):
    return df.applymap(lambda x: x/100 + 1).prod().apply(lambda x: (x - 1) * 100)

# Series showing the total change in price from day0 to today
totalPrice = total_percent_change(pricePercents)
totalVolume = total_percent_change(volumePercents)

def cumulative_percent_change(df: pd.DataFrame):
    return df.applymap(lambda x: x/100 + 1).cumprod().apply(lambda x: (x - 1) * 100)

# Cumulative hourly percent change if we had invested numDays ago:
cumuPrice = cumulative_percent_change(pricePercents)
cumuVolume = cumulative_percent_change(volumePercents)

In [98]:
# This class right here only exists because pandas dict(df) != df.to_list() 
# so to avoid different type handling in export_json, we made this.    
class Corrs(list):
    def to_dict(self):
        return dict(self)

# Prepares df.corr() as list for exporting
def list_corr(df):
    corrs = Corrs() # stores our result as a list of tuples for sorting at the end
    for i in range(1, df.shape[0]):
        for j in range(i):
            row = df.index[i]
            col = df.columns[j]
            corrs.append( (f"{row}-{col}", df.iloc[i, j]) )
            
    return Corrs( sorted(corrs, key=lambda x: x[1], reverse=True) )

# Export data to json files.
folder = 'static/json'
def export_json(d, filename, graphType):
    with open(f"{folder}/{filename}.json", 'w') as f:
        json.dump({'data': d.to_dict(), 'startDate': str(startDate), 'endDate': str(endDate), 
                   'numDays':numDays, 'excludedPairs': excluded_pairs, 'graphType': graphType} ,
                  f)
    return



export_json( cumuPrice , 'cumulative-price-percent', 'dygraph' )
export_json( cumuVolume , 'cumulative-volume-percent', 'dygraph' )

export_json( totalPrice , 'total-price-percent-change', 'hbar' )
export_json( totalVolume , 'total-volume-percent-change', 'hbar' )

export_json( prices , 'prices', 'dygraph' )
export_json( volumes , 'volumes', 'dygraph' )

export_json( pricePercents , 'percent-prices', 'dygraph' )
export_json( volumePercents , 'percent-volumes', 'dygraph' )

export_json( pricesLogReturn , 'log-return-prices', 'dygraph' )
export_json( volumesLogReturn , 'log-return-volumes', 'dygraph' )

export_json( list_corr( correlationVolumes ), 'correlation-volumes-pvp', 'hbar' )
export_json( list_corr( correlationPrices ), 'correlation-prices-pvp', 'hbar' )

export_json( list_corr( correlationVolumePercents ), 'correlation-volume-percents-pvp', 'hbar' )
export_json( list_corr( correlationPricePercents ), 'correlation-price-percents-pvp', 'hbar' )

export_json( list_corr(correlationVolumesLogReturn), 'correlation-volumes-log-return-pvp', 'hbar' )
export_json( list_corr(correlationPricesLogReturn), 'correlation-prices-log-return-pvp', 'hbar' )

export_json( correlationValues, 'correlation-volume-price', 'hbar' )
export_json( correlationPercents, 'correlation-volume-price-percents', 'hbar' )
export_json( correlationLogReturns, 'correlation-volume-price-log-returns', 'hbar' )


print(f'Done exporting files to /static/json')
print(f'Local Time: {datetime.now()}')

Done exporting files to /static/json
Local Time: 2019-01-11 07:35:48.917311


Algorithm for correlation of percent changes
log returns, instead of percent changes for correlation:

returns = pd.DataFrame(data={ col: np.log(prices[col]).diff() for col in prices}

In [ ]:
# coinlist = ["NANO","IOTA","XMR", "EOS","XRP","NEO","DASH"]
# # # coinlist = ["XRP"]
# re = lambda symbol: f"^{symbol}|{symbol}$"
# end = lambda symbol: f"{symbol}$"
# regstr = reduce(lambda x,y: x + "|" + re(y), [''] + coinlist)[1:]
# regstr